# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
! pip install gmaps
! pip install matplotlib
! pip install pandas
! pip install requests


In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
file = "weatherPy1.csv"

types_df = pd.read_csv(file)

df = types_df.drop(columns=["Unnamed: 0"])

df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,atuona,-9.8000,-139.0333,79.25,76,18,17.16,PF,1650830174
1,egvekinot,66.3167,-179.1667,17.37,87,21,4.34,RU,1650830214
2,faanui,-16.4833,-151.7500,81.30,70,14,6.85,PF,1650830215
3,tabou,4.4230,-7.3528,81.81,84,100,8.43,CI,1650830215
4,sisimiut,66.9395,-53.6735,27.48,89,95,3.60,GL,1650830215
...,...,...,...,...,...,...,...,...,...
1125,ulaanbaatar,47.9077,106.8832,37.53,60,75,11.18,MN,1650830247
1126,noyabrsk,63.1994,75.4507,16.86,93,100,4.74,RU,1650830327
1127,komsomolskiy,40.4272,71.7189,59.72,67,25,2.30,UZ,1650830374
1128,aripuana,-9.1667,-60.6333,86.34,63,76,1.39,BR,1650830374


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
df["Humidity"]

0       76
1       87
2       70
3       84
4       89
        ..
1125    60
1126    93
1127    67
1128    63
1129    89
Name: Humidity, Length: 1130, dtype: int64

In [22]:
gmaps.configure(api_key=g_key)

In [23]:
# locations

locations = df[["Lat", "Lng"]].astype(float)
humidity = df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=max(df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_df = df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_df = filtered_df.loc[(filtered_df["Max Temp"] < 80) & (filtered_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_df = filtered_df.loc[filtered_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_df = filtered_df.loc[filtered_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_df = filtered_df.dropna()

filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
153,castro,-24.7911,-50.0119,72.97,72,0,1.34,BR,1650830258
159,dawlatabad,36.4141,64.9053,70.59,49,0,2.57,AF,1650830260
196,arroio dos ratos,-30.0772,-51.7292,70.52,63,0,3.91,BR,1650830271
236,erzin,36.9559,36.2000,70.36,49,0,2.44,TR,1650830282
331,lumding,25.7535,93.1732,72.10,86,0,2.59,IN,1650830309
359,ridgecrest,35.6225,-117.6709,74.68,12,0,1.99,US,1650830292
376,pisco,-13.7000,-76.2167,70.02,82,0,6.91,PE,1650830322
396,arroio grande,-32.2375,-53.0869,75.31,35,0,9.80,BR,1650830327
718,castro,-24.7911,-50.0119,72.97,72,0,1.34,BR,1650830258
724,dawlatabad,36.4141,64.9053,70.59,49,0,2.57,AF,1650830260


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df = filtered_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 153: castro.
Closest hotel is CHACARA BAILLY.
------------
Retrieving Results for Index 159: dawlatabad.
Closest hotel is WASIM ASGAR KHAN.
------------
Retrieving Results for Index 196: arroio dos ratos.
Closest hotel is Hotel do Sul.
------------
Retrieving Results for Index 236: erzin.
Closest hotel is Artemis Otel.
------------
Retrieving Results for Index 331: lumding.
Closest hotel is Hotel Nilamber.
------------
Retrieving Results for Index 359: ridgecrest.
Closest hotel is SpringHill Suites by Marriott Ridgecrest.
------------
Retrieving Results for Index 376: pisco.
Closest hotel is Hotel San Isidro Oficial.
------------
Retrieving Results for Index 396: arroio grande.
Closest hotel is Motel Bela Vista.
------------
Retrieving Results for Index 718: castro.
Closest hotel is CHACARA BAILLY.
------------
Retrieving Results for Index 724: dawlatabad.
Closest hotel is WASIM ASGAR KHAN.
------------
Retrieving Results for Index 761: arroio dos ratos.
Cl

In [26]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
153,castro,-24.7911,-50.0119,72.97,72,0,1.34,BR,1650830258,CHACARA BAILLY
159,dawlatabad,36.4141,64.9053,70.59,49,0,2.57,AF,1650830260,WASIM ASGAR KHAN
196,arroio dos ratos,-30.0772,-51.7292,70.52,63,0,3.91,BR,1650830271,Hotel do Sul
236,erzin,36.9559,36.2000,70.36,49,0,2.44,TR,1650830282,Artemis Otel
331,lumding,25.7535,93.1732,72.10,86,0,2.59,IN,1650830309,Hotel Nilamber
359,ridgecrest,35.6225,-117.6709,74.68,12,0,1.99,US,1650830292,SpringHill Suites by Marriott Ridgecrest
376,pisco,-13.7000,-76.2167,70.02,82,0,6.91,PE,1650830322,Hotel San Isidro Oficial
396,arroio grande,-32.2375,-53.0869,75.31,35,0,9.80,BR,1650830327,Motel Bela Vista
718,castro,-24.7911,-50.0119,72.97,72,0,1.34,BR,1650830258,CHACARA BAILLY
724,dawlatabad,36.4141,64.9053,70.59,49,0,2.57,AF,1650830260,WASIM ASGAR KHAN


In [27]:
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))